Testing camera input


In [1]:
import cv2 as c
from mtcnn.mtcnn import MTCNN
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

Only if RTX card


In [2]:
phys = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(phys[0],True)


In [10]:
vc = c.VideoCapture(0)

f = False

mod_name = 'senet50'

if vc.isOpened():
    face_d = MTCNN()
    face_r = VGGFace(include_top=False, model=mod_name, input_shape=(224,224,3), pooling='avg')
    c.namedWindow("view")
    f, frame = vc.read()
    f_count = 0
    while f:
        pix = np.asarray(frame)
        faces = face_d.detect_faces(pix)
        face_pix = list()

        for face in faces:
            x1, y1, width, height = face['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            start = (x1,y1)
            end = (x2,y2)
            face_pix.append(pix[y1:y2, x1:x2])
            frame = c.rectangle(frame,start,end,(0,0,255),1)


        count = 0
        inp_features = list()
        for face in face_pix:
            face = c.resize(face, (224,224), interpolation=c.INTER_AREA)
            if mod_name == 'vgg16':
                inp_features.append(preprocess_input(np.asarray(face).astype('float64')))
            else:
                inp_features.append(preprocess_input(np.asarray(face).astype('float64'), version=2))
            c.imwrite('../data/test'+str(f_count)+'_'+str(count)+'.jpg',face)
            count += 1

        inp_features = np.asarray(inp_features)

        if len(inp_features) > 0:
            features = face_r.predict(inp_features)
            print(features)

        c.imshow("view",frame)
        f_count += 1
        f, frame = vc.read()
        key = c.waitKey(5)
        if key == 27:
            vc.release()
            c.destroyWindow("view")
            break


[[2.3828393e-02 4.6757981e-02 4.9419183e-02 ... 2.2229434e-03
  5.9891667e+00 7.4205868e-02]]
[[ 0.04550884  2.0849369   0.30369848 ...  0.01077637 10.078525
   0.04896839]]
[[0.04477264 0.9749889  0.5337436  ... 0.26893255 9.779438   0.05025182]]
[[ 0.04302651  2.0908766   0.4335049  ...  0.05871406 14.431076
   0.04748097]]
[[ 0.04116829  7.7305746   0.37873447 ...  0.01848643 14.775079
   0.04654349]]
[[ 0.04506614  9.726044    0.1795184  ...  0.03384791 16.090002
   0.0515501 ]]
